# Getting started with astir

## 0. Load necessary libraries

In [ ]:
!pip install -e ../../..
from astir.data_readers import from_csv_yaml
import pandas as pd

Obtaining file:///Users/kierancampbell/toronto/astir
  Attempting uninstall: astir
    Found existing installation: astir 0.0.1
    Uninstalling astir-0.0.1:
      Successfully uninstalled astir-0.0.1
  Running setup.py develop for astir


## 1. Load data

We start by reading expression data in the form of a csv file and marker gene information in the form of a yaml file:

In [ ]:
expression_mat_path = "../../../astir/tests/test-data/sce.csv"
yaml_marker_path = "../../../astir/tests/test-data/jackson-2020-markers.yml"

.. note:: 
    Expression data should already be cleaned and normalized, through e.g. a log transformation and winsorization.

We can view both the expression data and marker data:

In [ ]:
!head -n 20 ../../../astir/tests/test-data/jackson-2020-markers.yml

In [ ]:
pd.read_csv(expression_mat_path, index_col=0)[['EGFR','E-Cadherin', 'CD45', 'Cytokeratin 5']].head()

Then we can create an astir object using the `from_csv_yaml` function. For more data loading options, see the data loading tutorial.

In [ ]:
ast = from_csv_yaml(expression_mat_path, marker_yaml=yaml_marker_path)
print(ast)

## 2. Fitting cell types

To fit cell types, simply call

In [ ]:
ast.fit_type()

In [ ]:
# import sys
# from time import sleep
# from tqdm import tqdm

# for i in tqdm(range(20)):
#     sleep(0.1)

# values = range(20)
# with tqdm(total=len(values), file=sys.stdout) as pbar:
#     for i in values:
#         pbar.set_description('processed: %d' % (1 + i))
#         pbar.update(1)
#         sleep(0.1)

.. note:: 
    **Controlling inference**
    There are many different options for controlling inference in the `fit_type` function, including
    `max_epochs` (maximum number of epochs to train),
    `learning_rate` (ADAM optimizer learning rate),
    `batch_size` (minibatch size),
    `delta_loss` (stops iteration once the change in loss falls below this value),
    `n_inits` (number of restarts using random initializations).
    For full details, see the function documentation.

We can then get cell type assignment probabilities by calling

In [ ]:
assignments = ast.get_celltype_probabilities()
assignments

where each row corresponds to a cell, and each column to a cell type, with the entry being the probability of that cell belonging to a particular cell type.

To fetch an array corresponding to the most likely cell type assignments, call

In [ ]:
ast.get_celltypes()

## 3. Fitting cell state

Similarly as before, to fit cell state, call

In [ ]:
ast.fit_state()

and cell state assignments can be inferred via

In [ ]:
states = ast.get_cellstates()
states

## 4. Saving results

Both cell type and cell state information can easily be saved to disk via

In [ ]:
ast.type_to_csv("cell-types.csv")
ast.state_to_csv("cell-states.csv")

In [ ]:
!head -n 3 cell-types.csv

In [ ]:
!head -n 3 cell-states.csv

where the first (unnamed) column always corresponds to the cell name/ID.

## 5. Accessing internal functions and data

Data stored in `astir` objects is in the form of an `SCDataSet`. These can be retrieved via

In [ ]:
celltype_data = ast.get_type_dataset()
celltype_data

and similarly for cell state via `ast.get_state_dataset()`.

These have several helper functions to retrieve relevant information to the dataset:

In [ ]:
celltype_data.get_cells()[0:4] # cell names

In [ ]:
celltype_data.get_classes() # cell type names

In [ ]:
print(celltype_data.get_n_classes()) # number of cell types
print(celltype_data.get_n_features()) # number of features / proteins

In [ ]:
celltype_data.get_exprs() # Return a torch tensor corresponding to the expression data used